# Import librairies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering
from sklearn.metrics.cluster import adjusted_rand_score
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("les-arbres.csv", sep =';')

In [3]:
data.head()

,IDBASE,TYPEEMPLACEMENT,DOMANIALITE,ARRONDISSEMENT,COMPLEMENTADRESSE,NUMERO,LIEU / ADRESSE,IDEMPLACEMENT,LIBELLEFRANCAIS,GENRE,ESPECE,VARIETEOUCULTIVAR,CIRCONFERENCEENCM,HAUTEUR (m),STADEDEVELOPPEMENT,REMARQUABLE,geo_point_2d
0,2009192.0,Arbre,Alignement,PARIS 4E ARRDT,NaN,NaN,QUAI D ORLEANS,000202010,Peuplier,Populus,n. sp.,NaN,25.0,5.0,J,0.0,"48.85148173,2.354487895"
1,111900.0,Arbre,Jardin,PARIS 20E ARRDT,NaN,NaN,SQUARE DE LA PAIX / 128 RUE DE LAGNY,000105095,Frêne à fleurs,Fraxinus,ornus,NaN,30.0,5.0,J,0.0,"48.8484840426,2.41578924906"
2,128083.0,Arbre,Jardin,PARIS 19E ARRDT,NaN,NaN,SQUARE DE LA RUE COMPANS,00000017,Robinier,Robinia,pseudoacacia,NaN,130.0,6.0,NaN,NaN,"48.8780600881,2.39211203593"
3,208933.0,Arbre,Alignement,PARIS 15E ARRDT,35,NaN,QUAI DE GRENELLE,000101002,Marronnier,Aesculus,hippocastanum,''Baumannii'',150.0,10.0,A,0.0,"48.8523539904,2.28610910036"
4,224733.0,Arbre,Alignement,PARIS 8E ARRDT,NaN,NaN,PLACE CHARLES DE GAULLE,001104004,Platane,Platanus,x hispanica,NaN,67.0,8.0,A,0.0,"48.8734818781,2.29646123625"


In [4]:
data.shape

(204134, 17)

In [5]:
data.describe()

,IDBASE,NUMERO,CIRCONFERENCEENCM,HAUTEUR (m),REMARQUABLE
count,2.041340e+05,0.0,204134.000000,204134.000000,163805.000000
mean,4.331874e+05,NaN,85.056933,13.296624,0.001111
std,6.018687e+05,NaN,805.651088,1951.825029,0.033314
min,9.987400e+04,NaN,0.000000,0.000000,0.000000
25%,1.576022e+05,NaN,34.000000,5.000000,0.000000
50%,2.244995e+05,NaN,70.000000,8.000000,0.000000
75%,2.786508e+05,NaN,115.000000,12.000000,0.000000
max,2.030899e+06,NaN,250255.000000,881818.000000,1.000000


In [6]:
data.dtypes

IDBASE                float64
TYPEEMPLACEMENT        object
DOMANIALITE            object
ARRONDISSEMENT         object
COMPLEMENTADRESSE      object
NUMERO                float64
LIEU / ADRESSE         object
IDEMPLACEMENT          object
LIBELLEFRANCAIS        object
GENRE                  object
ESPECE                 object
VARIETEOUCULTIVAR      object
CIRCONFERENCEENCM     float64
HAUTEUR (m)           float64
STADEDEVELOPPEMENT     object
REMARQUABLE           float64
geo_point_2d           object
dtype: object

In [7]:
data.isna().sum()

IDBASE                     0
TYPEEMPLACEMENT            0
DOMANIALITE                0
ARRONDISSEMENT             0
COMPLEMENTADRESSE     167631
NUMERO                204134
LIEU / ADRESSE             0
IDEMPLACEMENT              0
LIBELLEFRANCAIS          710
GENRE                      1
ESPECE                  2044
VARIETEOUCULTIVAR     166632
CIRCONFERENCEENCM          0
HAUTEUR (m)                0
STADEDEVELOPPEMENT     57073
REMARQUABLE            40329
geo_point_2d               0
dtype: int64

In [8]:
data.COMPLEMENTADRESSE.isna().sum()

167631

In [9]:
x = data.COMPLEMENTADRESSE.isna().sum()/data.shape[0]*100
x

82.11811849079527

In [10]:
x = data.GENRE.isna().sum()/data.shape[0]*100
x

0.0004898742982550677

In [11]:
x = data.NUMERO.isna().sum()/data.shape[0]*100
x

100.0

In [12]:
x = data.ESPECE.isna().sum()/data.shape[0]*100
x

1.0013030656333586

In [13]:
x = data.LIBELLEFRANCAIS.isna().sum()/data.shape[0]*100
x

0.3478107517610981

In [14]:
x = data.STADEDEVELOPPEMENT.isna().sum()/data.shape[0]*100
x

27.958595824311484

In [15]:
x = data.VARIETEOUCULTIVAR.isna().sum()/data.shape[0]*100
x

81.62873406683845

In [16]:
x = data.REMARQUABLE.isna().sum()/data.shape[0]*100
x

19.756140574328626

In [17]:
data_drop = data.drop(columns=['COMPLEMENTADRESSE', 'NUMERO','VARIETEOUCULTIVAR'])

In [18]:
data_drop.head()

,IDBASE,TYPEEMPLACEMENT,DOMANIALITE,ARRONDISSEMENT,LIEU / ADRESSE,IDEMPLACEMENT,LIBELLEFRANCAIS,GENRE,ESPECE,CIRCONFERENCEENCM,HAUTEUR (m),STADEDEVELOPPEMENT,REMARQUABLE,geo_point_2d
0,2009192.0,Arbre,Alignement,PARIS 4E ARRDT,QUAI D ORLEANS,000202010,Peuplier,Populus,n. sp.,25.0,5.0,J,0.0,"48.85148173,2.354487895"
1,111900.0,Arbre,Jardin,PARIS 20E ARRDT,SQUARE DE LA PAIX / 128 RUE DE LAGNY,000105095,Frêne à fleurs,Fraxinus,ornus,30.0,5.0,J,0.0,"48.8484840426,2.41578924906"
2,128083.0,Arbre,Jardin,PARIS 19E ARRDT,SQUARE DE LA RUE COMPANS,00000017,Robinier,Robinia,pseudoacacia,130.0,6.0,NaN,NaN,"48.8780600881,2.39211203593"
3,208933.0,Arbre,Alignement,PARIS 15E ARRDT,QUAI DE GRENELLE,000101002,Marronnier,Aesculus,hippocastanum,150.0,10.0,A,0.0,"48.8523539904,2.28610910036"
4,224733.0,Arbre,Alignement,PARIS 8E ARRDT,PLACE CHARLES DE GAULLE,001104004,Platane,Platanus,x hispanica,67.0,8.0,A,0.0,"48.8734818781,2.29646123625"


In [19]:
data_drop.dropna(inplace=True)

In [20]:
data_drop.head()

,IDBASE,TYPEEMPLACEMENT,DOMANIALITE,ARRONDISSEMENT,LIEU / ADRESSE,IDEMPLACEMENT,LIBELLEFRANCAIS,GENRE,ESPECE,CIRCONFERENCEENCM,HAUTEUR (m),STADEDEVELOPPEMENT,REMARQUABLE,geo_point_2d
0,2009192.0,Arbre,Alignement,PARIS 4E ARRDT,QUAI D ORLEANS,000202010,Peuplier,Populus,n. sp.,25.0,5.0,J,0.0,"48.85148173,2.354487895"
1,111900.0,Arbre,Jardin,PARIS 20E ARRDT,SQUARE DE LA PAIX / 128 RUE DE LAGNY,000105095,Frêne à fleurs,Fraxinus,ornus,30.0,5.0,J,0.0,"48.8484840426,2.41578924906"
3,208933.0,Arbre,Alignement,PARIS 15E ARRDT,QUAI DE GRENELLE,000101002,Marronnier,Aesculus,hippocastanum,150.0,10.0,A,0.0,"48.8523539904,2.28610910036"
4,224733.0,Arbre,Alignement,PARIS 8E ARRDT,PLACE CHARLES DE GAULLE,001104004,Platane,Platanus,x hispanica,67.0,8.0,A,0.0,"48.8734818781,2.29646123625"
5,2003211.0,Arbre,Alignement,BOIS DE VINCENNES,CARREFOUR DE LA FERME DE LA FAISANDERIE,000202001,Pommier,Malus,n. sp.,25.0,5.0,J,0.0,"48.8236992293,2.45805540839"


In [21]:
data_drop.isna().sum()

IDBASE                0
TYPEEMPLACEMENT       0
DOMANIALITE           0
ARRONDISSEMENT        0
LIEU / ADRESSE        0
IDEMPLACEMENT         0
LIBELLEFRANCAIS       0
GENRE                 0
ESPECE                0
CIRCONFERENCEENCM     0
HAUTEUR (m)           0
STADEDEVELOPPEMENT    0
REMARQUABLE           0
geo_point_2d          0
dtype: int64

In [22]:
data_drop.shape

(143335, 14)

In [23]:
data_drop.TYPEEMPLACEMENT.value_counts()

Arbre    143335
Name: TYPEEMPLACEMENT, dtype: int64

In [24]:
data_drop.drop(columns = ['TYPEEMPLACEMENT'], inplace = True)

In [25]:
data_drop.DOMANIALITE.value_counts()

Alignement      98195
Jardin          20811
CIMETIERE       12359
DASCO            5531
DJS              2979
PERIPHERIQUE     2290
DFPE             1154
DAC                16
Name: DOMANIALITE, dtype: int64

In [26]:
data_drop.ARRONDISSEMENT.value_counts()

PARIS 16E ARRDT      13755
PARIS 20E ARRDT      12428
PARIS 13E ARRDT      12355
PARIS 15E ARRDT      10043
PARIS 14E ARRDT      10037
PARIS 19E ARRDT       9249
PARIS 12E ARRDT       8933
PARIS 18E ARRDT       8749
BOIS DE VINCENNES     8735
PARIS 7E ARRDT        7963
PARIS 17E ARRDT       7949
PARIS 8E ARRDT        6140
PARIS 11E ARRDT       4465
BOIS DE BOULOGNE      3851
VAL-DE-MARNE          3601
SEINE-SAINT-DENIS     3155
PARIS 10E ARRDT       2527
PARIS 4E ARRDT        2127
PARIS 5E ARRDT        1795
PARIS 6E ARRDT        1413
PARIS 1ER ARRDT       1074
PARIS 9E ARRDT        1041
PARIS 3E ARRDT         988
HAUTS-DE-SEINE         494
PARIS 2E ARRDT         468
Name: ARRONDISSEMENT, dtype: int64

In [27]:
data_drop['LIEU / ADRESSE'].value_counts()

PARC FLORAL DE PARIS / ROUTE DE LA PYRAMIDE             1644
INSEP / AVENUE DU TREMBLAY                              1283
AVENUE DES CHAMPS ELYSEES                               1230
JARDIN DU CHAMP DE MARS                                 1075
PARC MONTSOURIS / 28 BOULEVARD JOURDAN                  1043
                                                        ... 
CIMETIERE DE PANTIN / AVENUE DES GARDES / DIV 166          1
CITE TRAEGER                                               1
CIMETIERE DE BAGNEUX / DIV 41                              1
CIMETIERE DE THIAIS / AVENUE E / DIV 61                    1
CIMETIERE DE PANTIN / AVENUE DES MERISIERS / DIV 128       1
Name: LIEU / ADRESSE, Length: 4977, dtype: int64

In [28]:
data_drop.drop(columns = ['LIEU / ADRESSE','ESPECE'], inplace = True)

In [29]:
data_drop['LIBELLEFRANCAIS'].value_counts()

Platane                 38508
Marronnier              20882
Tilleul                 16105
Erable                  10810
Sophora                  9774
                        ...  
Euscaphis                   1
Fremontia                   1
Kalopanax                   1
Heptacodion de Chine        1
Andromède                   1
Name: LIBELLEFRANCAIS, Length: 174, dtype: int64

In [30]:
data_drop['LIBELLEFRANCAIS'].value_counts()[:20]

Platane                 38508
Marronnier              20882
Tilleul                 16105
Erable                  10810
Sophora                  9774
Micocoulier              3427
Frêne                    2815
Noisetier de Byzance     2662
Chêne                    2583
Poirier à fleurs         2491
Cerisier à fleurs        2259
Peuplier                 2168
Pin                      2069
Charme                   1944
Robinier                 1426
Paulownia                1078
Fevier                    977
If                        962
Bouleau                   921
Cedrele                   812
Name: LIBELLEFRANCAIS, dtype: int64

In [31]:
top20 = data_drop['LIBELLEFRANCAIS'].value_counts().index.tolist()[20:]
top20

['Arbre aux quarante écus',
 'Noyer',
 'Arbre de Judée',
 'Orme',
 'Magnolia',
 'Prunier à fleurs',
 'Pterocarya',
 'Frêne à fleurs',
 'Tulipier',
 'Ailante',
 'Hêtre',
 'Mûrier',
 'Merisier',
 'Pommier à fleurs',
 'Cèdre',
 'Cyprès',
 'Savonnier',
 'Charme-Houblon',
 'Copalme',
 'Lilas des indes',
 'Catalpa',
 'Pommier à fruits',
 'Cerisier à grappes',
 'Troene',
 'Prunus n. sp.',
 'Aulne',
 'Orme de Sibérie',
 'Faux-cyprès',
 'Amélanchier',
 'Pommier',
 'Non spécifié',
 'Parrotie de Perse - Arbre de fer',
 'Houx',
 'Thuya',
 'Cornouiller',
 'Alisier',
 'Chicot du Canada',
 'Aubépine',
 'Arbre à soie',
 'Poirier à fruits',
 'Châtaignier',
 'Palmier',
 'Sorbier',
 'Saule',
 'Epicéa',
 'Eleagnus - Chalef',
 'Lilas de Perse',
 'Plaqueminier',
 'Aubepine',
 'Laurier du Portugal',
 'Poirier',
 'Olivier',
 'Photinia',
 'Prunier à fruits',
 'Troëne',
 'Metaséquoia',
 'Noisetier',
 'Sequoia',
 'Pêcher',
 'Figuier',
 'Amandier',
 'Sureau',
 'Sapin',
 'Arbre à caramel',
 'Cerisier à fruits',
 '

In [32]:
data_drop['LIBELLEFRANCAIS'][data_drop['LIBELLEFRANCAIS'].isin(top20)] = "OTHER"

/Users/julieG/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
data_drop['LIBELLEFRANCAIS'].value_counts()[0:20]

Platane                 38508
Marronnier              20882
OTHER                   18662
Tilleul                 16105
Erable                  10810
Sophora                  9774
Micocoulier              3427
Frêne                    2815
Noisetier de Byzance     2662
Chêne                    2583
Poirier à fleurs         2491
Cerisier à fleurs        2259
Peuplier                 2168
Pin                      2069
Charme                   1944
Robinier                 1426
Paulownia                1078
Fevier                    977
If                        962
Bouleau                   921
Name: LIBELLEFRANCAIS, dtype: int64

In [34]:
data_drop['GENRE'].value_counts()[:20]

Platanus     38508
Aesculus     20882
Tilia        16105
Acer         10810
Sophora       9774
Prunus        4455
Celtis        3427
Fraxinus      3414
Pyrus         2724
Corylus       2717
Quercus       2582
Populus       2168
Pinus         2069
Carpinus      1944
Robinia       1426
Paulownia     1078
Malus         1071
Gleditsia      974
Taxus          962
Betula         921
Name: GENRE, dtype: int64

In [35]:
top20_genre = data_drop['GENRE'].value_counts().index.tolist()[20:]
top20_genre

['Toona',
 'Ginkgo',
 'Juglans',
 'Cercis',
 'Ulmus',
 'Magnolia',
 'Pterocarya',
 'Liriodendron',
 'Ailanthus',
 'Fagus',
 'Cedrus',
 'Morus',
 'Koelreuteria',
 'Ostrya',
 'Liquidambar',
 'Lagerstroemia',
 'Cupressus',
 'Ligustrum',
 'Catalpa',
 'Sorbus',
 'Alnus',
 'Crataegus',
 'Zelkova',
 'Chamaecyparis',
 'Amelanchier',
 'Non spécifié',
 'Parrotia',
 'Ilex',
 'Cornus',
 'Gymnocladus',
 'Thuja',
 'Albizia',
 'Castanea',
 'Trachycarpus',
 'x Cupressocyparis',
 'Salix',
 'Elaeagnus',
 'Picea',
 'Melia',
 'Diospyros',
 'Broussonetia',
 'Olea',
 'Photinia',
 'Metasequoia',
 'Ficus',
 'Sambucus',
 'Abies',
 'Cercidiphyllum',
 'Cydonia',
 'Calocedrus',
 'Cladrastis',
 'Laburnum',
 'Eucalyptus',
 'Tetradium',
 'Maclura',
 'Buxus',
 'Sequoiadendron',
 'Sequoia',
 'Clerodendrum',
 'Laurus',
 'x Chitalpa',
 'Fontanesia',
 'Platycladus',
 'Mespilus',
 'Ehretia',
 'Phellodendron',
 'Hibiscus',
 'Zanthoxylum',
 'Larix',
 'Eriobotrya',
 'Araucaria',
 'Phillyrea',
 'Davidia',
 'Euonymus',
 'Tamar

In [36]:
data_drop['GENRE'][data_drop['GENRE'].isin(top20_genre)] = "OTHER"

/Users/julieG/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
data_drop['GENRE'].value_counts()

Platanus     38508
Aesculus     20882
Tilia        16105
OTHER        15324
Acer         10810
Sophora       9774
Prunus        4455
Celtis        3427
Fraxinus      3414
Pyrus         2724
Corylus       2717
Quercus       2582
Populus       2168
Pinus         2069
Carpinus      1944
Robinia       1426
Paulownia     1078
Malus         1071
Gleditsia      974
Taxus          962
Betula         921
Name: GENRE, dtype: int64

In [38]:
data_drop['STADEDEVELOPPEMENT'].value_counts()

A     69873
JA    36174
J     30282
M      7006
Name: STADEDEVELOPPEMENT, dtype: int64

In [39]:
data_drop['REMARQUABLE'].value_counts()

0.0    143154
1.0       181
Name: REMARQUABLE, dtype: int64

In [40]:
data_drop.drop(columns = ['IDEMPLACEMENT'], inplace = True)

In [41]:
data_drop.dtypes

IDBASE                float64
DOMANIALITE            object
ARRONDISSEMENT         object
LIBELLEFRANCAIS        object
GENRE                  object
CIRCONFERENCEENCM     float64
HAUTEUR (m)           float64
STADEDEVELOPPEMENT     object
REMARQUABLE           float64
geo_point_2d           object
dtype: object

In [54]:
data_drop.head()

,IDBASE,DOMANIALITE,ARRONDISSEMENT,LIBELLEFRANCAIS,GENRE,CIRCONFERENCEENCM,HAUTEUR (m),STADEDEVELOPPEMENT,REMARQUABLE,geo_point_2d
0,2009192.0,Alignement,PARIS 4E ARRDT,Peuplier,Populus,25.0,5.0,J,0.0,"48.85148173,2.354487895"
1,111900.0,Jardin,PARIS 20E ARRDT,OTHER,Fraxinus,30.0,5.0,J,0.0,"48.8484840426,2.41578924906"
3,208933.0,Alignement,PARIS 15E ARRDT,Marronnier,Aesculus,150.0,10.0,A,0.0,"48.8523539904,2.28610910036"
4,224733.0,Alignement,PARIS 8E ARRDT,Platane,Platanus,67.0,8.0,A,0.0,"48.8734818781,2.29646123625"
5,2003211.0,Alignement,BOIS DE VINCENNES,OTHER,Malus,25.0,5.0,J,0.0,"48.8236992293,2.45805540839"


In [42]:
#object_cols = data_drop.columns.values[data_drop.dtypes == object]
data_dummy = pd.get_dummies(data_drop, columns=['DOMANIALITE','ARRONDISSEMENT','LIBELLEFRANCAIS','GENRE','STADEDEVELOPPEMENT'], drop_first=True)

In [43]:
data_dummy.head()

,IDBASE,CIRCONFERENCEENCM,HAUTEUR (m),REMARQUABLE,geo_point_2d,DOMANIALITE_CIMETIERE,DOMANIALITE_DAC,DOMANIALITE_DASCO,DOMANIALITE_DFPE,DOMANIALITE_DJS,...,GENRE_Prunus,GENRE_Pyrus,GENRE_Quercus,GENRE_Robinia,GENRE_Sophora,GENRE_Taxus,GENRE_Tilia,STADEDEVELOPPEMENT_J,STADEDEVELOPPEMENT_JA,STADEDEVELOPPEMENT_M
0,2009192.0,25.0,5.0,0.0,"48.85148173,2.354487895",0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,111900.0,30.0,5.0,0.0,"48.8484840426,2.41578924906",0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,208933.0,150.0,10.0,0.0,"48.8523539904,2.28610910036",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,224733.0,67.0,8.0,0.0,"48.8734818781,2.29646123625",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2003211.0,25.0,5.0,0.0,"48.8236992293,2.45805540839",0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [44]:
data_dummy.dtypes

IDBASE                   float64
CIRCONFERENCEENCM        float64
HAUTEUR (m)              float64
REMARQUABLE              float64
geo_point_2d              object
                          ...   
GENRE_Taxus                uint8
GENRE_Tilia                uint8
STADEDEVELOPPEMENT_J       uint8
STADEDEVELOPPEMENT_JA      uint8
STADEDEVELOPPEMENT_M       uint8
Length: 79, dtype: object

In [53]:
data_dummy[data_dummy.columns[0:5]]

,IDBASE,CIRCONFERENCEENCM,HAUTEUR (m),REMARQUABLE,geo_point_2d
0,2009192.0,25.0,5.0,0.0,"48.85148173,2.354487895"
1,111900.0,30.0,5.0,0.0,"48.8484840426,2.41578924906"
3,208933.0,150.0,10.0,0.0,"48.8523539904,2.28610910036"
4,224733.0,67.0,8.0,0.0,"48.8734818781,2.29646123625"
5,2003211.0,25.0,5.0,0.0,"48.8236992293,2.45805540839"
...,...,...,...,...,...
204127,236944.0,40.0,6.0,0.0,"48.8558457104,2.34092534027"
204128,243074.0,150.0,15.0,0.0,"48.8743197345,2.37750236537"
204129,106203.0,110.0,11.0,0.0,"48.8540221889,2.30211517789"
204131,160498.0,165.0,10.0,0.0,"48.8372086759,2.32732312804"


In [58]:
data_dummy.drop(columns='geo_point_2d', inplace=True)

In [59]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(data_dummy[data_dummy.columns[1:]])

data_scale = pd.DataFrame(scaler.transform(data_dummy[data_dummy.columns[1:]]), columns=data_dummy.columns[1:])

In [47]:
#data_scale.head()

In [48]:
#model = KMeans()
#visualizer = KElbowVisualizer(model, k=(2,12), metric='silhouette')
#visualizer.fit(data_scale)
#visualizer.poof()

In [49]:
#data_drop.drop(columns = ['IDBASE'], inplace=True)

In [50]:
#data_drop.head()

In [61]:
data_scale.head()

,CIRCONFERENCEENCM,HAUTEUR (m),REMARQUABLE,DOMANIALITE_CIMETIERE,DOMANIALITE_DAC,DOMANIALITE_DASCO,DOMANIALITE_DFPE,DOMANIALITE_DJS,DOMANIALITE_Jardin,DOMANIALITE_PERIPHERIQUE,...,GENRE_Prunus,GENRE_Pyrus,GENRE_Quercus,GENRE_Robinia,GENRE_Sophora,GENRE_Taxus,GENRE_Tilia,STADEDEVELOPPEMENT_J,STADEDEVELOPPEMENT_JA,STADEDEVELOPPEMENT_M
0,-0.079549,-0.005240,-0.035558,-0.307182,-0.010566,-0.200341,-0.090091,-0.145687,-0.412132,-0.12742,...,-0.179103,-0.139185,-0.135441,-0.100243,-0.270518,-0.0822,-0.355783,1.932185,-0.581005,-0.226694
1,-0.074341,-0.005240,-0.035558,-0.307182,-0.010566,-0.200341,-0.090091,-0.145687,2.426410,-0.12742,...,-0.179103,-0.139185,-0.135441,-0.100243,-0.270518,-0.0822,-0.355783,1.932185,-0.581005,-0.226694
2,0.050634,-0.003094,-0.035558,-0.307182,-0.010566,-0.200341,-0.090091,-0.145687,-0.412132,-0.12742,...,-0.179103,-0.139185,-0.135441,-0.100243,-0.270518,-0.0822,-0.355783,-0.517549,-0.581005,-0.226694
3,-0.035807,-0.003952,-0.035558,-0.307182,-0.010566,-0.200341,-0.090091,-0.145687,-0.412132,-0.12742,...,-0.179103,-0.139185,-0.135441,-0.100243,-0.270518,-0.0822,-0.355783,-0.517549,-0.581005,-0.226694
4,-0.079549,-0.005240,-0.035558,-0.307182,-0.010566,-0.200341,-0.090091,-0.145687,-0.412132,-0.12742,...,-0.179103,-0.139185,-0.135441,-0.100243,-0.270518,-0.0822,-0.355783,1.932185,-0.581005,-0.226694


In [ ]:
data_scale.drop(columns = ['Cluster2'], inplace=True)

In [60]:
model = KMeans()
model.fit(data_scale)
model_2 = KMeans(n_clusters = 5)
model_2.fit(data_scale)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
#data_scale['Cluster2'] = model_2.predict(data_scale)

In [63]:
data_drop['Cluster2'] = model_2.predict(data_scale)

In [62]:
data_drop['Cluster'] = model.predict(data_scale)

In [65]:
data_drop.to_csv('data_clusters.csv')

In [64]:
data_drop.head()

,IDBASE,DOMANIALITE,ARRONDISSEMENT,LIBELLEFRANCAIS,GENRE,CIRCONFERENCEENCM,HAUTEUR (m),STADEDEVELOPPEMENT,REMARQUABLE,geo_point_2d,Cluster,Cluster2
0,2009192.0,Alignement,PARIS 4E ARRDT,Peuplier,Populus,25.0,5.0,J,0.0,"48.85148173,2.354487895",7,0
1,111900.0,Jardin,PARIS 20E ARRDT,OTHER,Fraxinus,30.0,5.0,J,0.0,"48.8484840426,2.41578924906",3,0
3,208933.0,Alignement,PARIS 15E ARRDT,Marronnier,Aesculus,150.0,10.0,A,0.0,"48.8523539904,2.28610910036",0,2
4,224733.0,Alignement,PARIS 8E ARRDT,Platane,Platanus,67.0,8.0,A,0.0,"48.8734818781,2.29646123625",7,0
5,2003211.0,Alignement,BOIS DE VINCENNES,OTHER,Malus,25.0,5.0,J,0.0,"48.8236992293,2.45805540839",3,0


In [ ]:
#model.fit(data_scale)
#data_scale['Cluster'] = model.predict(data_scale)
#data_drop['Cluster'] = model.predict(data_scale)
#sns.pairplot(data_scale, hue='Cluster', vars=data_scale.columns[:-1])

In [ ]:
#data_scale['Cluster'] = model.predict(data_scale)

In [ ]:
#data_drop['Cluster'] = model.predict(data_scale)

In [66]:
pca = PCA()
pca.fit(data_scale)
pca_x = pca.transform(data_scale)

In [67]:
df = pd.DataFrame([pca_x[:, 0], pca_x[:, 1]]).T
df.columns = ['PC1', 'PC2']
df = pd.concat([df, data_scale['Cluster']], axis=1)
fig = sns.lmplot('PC1', 'PC2', data=df, hue='Cluster', fit_reg=False)

KeyError: 'Cluster'

In [ ]:
#fig.savefig('figurePCAV2.pdf')

In [ ]:
"""df_2 = pd.DataFrame([pca_x[:, 0], pca_x[:, 1]]).T
df_2.columns = ['PC1', 'PC2']
df_2 = pd.concat([df_2, data_scale['Cluster2']], axis=1)
fig_2 = sns.lmplot('PC1', 'PC2', data=df_2, hue='Cluster2', fit_reg=False)"""

In [ ]:
#fig_2.savefig('figurePCAV3.pdf')

In [ ]:
#data_scale.head()

In [ ]:
#data_drop.head()

In [ ]:
#data_drop.to_csv('Cluster.csv')

In [ ]:
"""cluster0 = data_drop[data_drop.Cluster == 0]
cluster1 = data_drop[data_drop.Cluster == 1]
cluster2 = data_drop[data_drop.Cluster == 2]
cluster3 = data_drop[data_drop.Cluster == 3]
cluster4 = data_drop[data_drop.Cluster == 4]
cluster5 = data_drop[data_drop.Cluster == 5]
cluster6 = data_drop[data_drop.Cluster == 6]
cluster7 = data_drop[data_drop.Cluster == 7]
cluster8 = data_drop[data_drop.Cluster == 8]
cluster9 = data_drop[data_drop.Cluster == 9]
cluster10 = data_drop[data_drop.Cluster == 10]"""

In [ ]:
#l_clusters = [data_drop[data_drop.Cluster == i] for i in range(0,11)]

In [ ]:
#[df_.shape[0] for df_ in l_clusters]

In [ ]:
#subset[column].dtype == "int32"

In [ ]:
"""for column in data_drop.columns:
    for cluster, subset in enumerate(l_clusters):
        if subset[column].dtype == "object":
            print(cluster)
            print(column)
            print(subset.shape[0])
            print(subset[column].value_counts(normalize=True))"""
        
        

        

In [ ]:
"""for column in data_drop.columns:
    for cluster, subset in enumerate(l_clusters):
        if subset[column].dtype != "object":
            plt.title(str(cluster) +"-" + column + "-" + str(subset.shape[0]) )
            print(subset[column].max())
#           print(column)
            print(subset.shape[0])
            subset[column].hist()
            plt.show()
        """
        

        

In [ ]:
#print(data_drop["CIRCONFERENCEENCM"].sort_values(ascending=False).tolist())

In [ ]:
#data_drop["CIRCONFERENCEENCM"].describe()

In [ ]:
#len([250255.0, 172180.0, 160170.0, 110120.0, 30140.0, 20130.0, 9130.0, 9035.0, 8107.0, 6125.0, 6085.0, 5545.0, 4090.0, 3120.0, 2750.0, 2545.0, 2260.0, 2219.0, 2196.0, 2090.0, 2025.0, 1801.0, 1590.0, 1360.0, 1345.0, 1340.0, 1337.0, 1240.0, 1240.0, 1230.0, 1220.0, 1200.0, 1200.0, 1190.0, 1190.0, 1160.0, 1160.0, 1160.0, 1155.0, 1151.0, 1150.0, 1145.0, 1140.0, 1140.0, 1140.0, 1138.0, 1133.0, 1130.0, 1124.0, 1120.0, 1120.0, 1112.0, 1106.0, 1102.0, 1054.0, 1005.0, 1000.0, 998.0, 784.0])

In [ ]:
#print(data_drop["HAUTEUR (m)"].sort_values(ascending=False).tolist())

In [ ]:
#data_drop["HAUTEUR (m)"].describe()

In [ ]:
#data_drop[data_drop["HAUTEUR (m)"]==150]